## Inisialisasi Library untuk Tahap Inference

Pada tahap inference atau prediksi, kita perlu memuat beberapa library penting yang dibutuhkan untuk membaca model, melakukan preprocessing input, dan mengatur eksekusi program. Berikut penjelasan masing-masing library:

- `pickle`: Digunakan untuk membuka file hasil serialisasi model dan tokenizer yang sebelumnya disimpan dalam format `.pkl`.
- `numpy`: Digunakan untuk manipulasi data numerik seperti array, yang umum digunakan dalam pengolahan data machine learning.
- `tensorflow`: Digunakan untuk memuat dan menjalankan model deep learning, khususnya model LSTM.
- `pad_sequences` dari Keras: Digunakan untuk menyamakan panjang input teks agar dapat diproses oleh model LSTM.
- `warnings`: Digunakan untuk menonaktifkan peringatan runtime agar output lebih bersih dan mudah dibaca.

In [1]:
# Mengimpor library untuk membaca file model dan tokenizer yang disimpan
import pickle

# Mengimpor library untuk manipulasi array dan operasi numerik
import numpy as np

# Mengimpor TensorFlow untuk memuat dan menjalankan model deep learning
import tensorflow as tf

# Mengimpor fungsi untuk padding input teks agar memiliki panjang yang seragam
from keras.preprocessing.sequence import pad_sequences

# Menonaktifkan peringatan agar output lebih bersih
import warnings
warnings.filterwarnings("ignore")

## Memuat Model dan Alat Preprocessing untuk Inference

Pada tahap ini, kita akan memuat ulang seluruh model serta alat bantu preprocessing yang sebelumnya telah disimpan di folder `../Models`. Ini penting dilakukan sebelum melakukan inference (prediksi) pada data baru.

Berikut komponen yang dimuat:

- **Model LSTM**: File `.h5` berisi model neural network LSTM untuk klasifikasi sentimen berbasis word embedding.
- **Model SVM**: File `.pkl` hasil serialisasi model Support Vector Machine yang telah dilatih.
- **Model Random Forest**: File `.pkl` yang berisi model ensemble Random Forest.
- **TF-IDF Vectorizer**: Digunakan untuk mentransformasi teks menjadi fitur numerik berbasis frekuensi kata.
- **Tokenizer**: Untuk mengubah teks mentah menjadi urutan token numerik yang dapat digunakan oleh model LSTM.

Semua file ini disimpan di direktori `../Models` dan dibaca ulang ke memori agar siap digunakan untuk proses klasifikasi data baru.

In [6]:
# Memuat model LSTM dari file .h5 yang berada di dalam folder ../Models
lstm_model = tf.keras.models.load_model("../Models/lstm_sentiment_model.h5")

# Membuka dan memuat model SVM dari file pickle
with open("../Models/svm_sentiment_model.pkl", "rb") as f:
    svm_model = pickle.load(f)

# Membuka dan memuat model Random Forest dari file pickle
with open("../Models/rf_sentiment_model.pkl", "rb") as f:
    rf_model = pickle.load(f)

# Memuat kembali TF-IDF Vectorizer untuk mentransformasi teks ke bentuk vektor numerik
with open("../Models/vectorizer.pkl", "rb") as f:
    vectorizer = pickle.load(f)

# Memuat tokenizer untuk mengubah teks menjadi urutan token angka untuk model LSTM
with open("../Models/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

## Preprocessing Untuk Data Baru

Pada tahap ini, kita menyiapkan sejumlah data teks baru yang akan digunakan untuk pengujian model klasifikasi sentimen yang telah dilatih sebelumnya. Data ini akan diproses agar sesuai dengan format input masing-masing model:

1. **Data Teks Baru**: Terdiri dari tiga kalimat dengan sentimen yang bervariasi (positif, negatif, dan netral).
2. **Untuk Model LSTM**:
   - Teks diubah menjadi urutan token menggunakan `Tokenizer`.
   - Urutan token dipadatkan (padded) agar panjangnya seragam sesuai panjang maksimum yang digunakan saat pelatihan.
3. **Untuk Model SVM dan Random Forest**:
   - Teks diubah menjadi representasi vektor numerik menggunakan TF-IDF vectorizer yang telah dilatih.

Setelah tahap ini, data siap digunakan untuk melakukan prediksi sentimen dengan masing-masing model.

In [7]:
# Data teks baru yang akan digunakan untuk prediksi sentimen
new_texts = [
    "Produk ini sangat bagus!",  # Kemungkinan sentimen positif
    "Pelayanan sangat lambat dan mengecewakan.",  # Kemungkinan sentimen negatif
    "Harganya cukup mahal tapi kualitasnya biasa saja."  # Kemungkinan netral
]

# Preprocessing untuk Model LSTM:
# Mengubah teks menjadi urutan token numerik
new_sequences = tokenizer.texts_to_sequences(new_texts)

# Melakukan padding pada urutan token agar seragam panjangnya (maxlen=100)
new_padded = pad_sequences(new_sequences, maxlen=100)

# Preprocessing untuk Model SVM dan Random Forest:
# Mengubah teks menjadi representasi vektor numerik dengan metode TF-IDF
new_tfidf = vectorizer.transform(new_texts)

## Perform Sentiment Prediction

Setelah data baru selesai diproses, langkah berikutnya adalah melakukan prediksi sentimen menggunakan tiga model berbeda yang telah dilatih sebelumnya, yaitu LSTM, SVM, dan Random Forest. 

Berikut penjelasan masing-masing langkah:

1. **Model LSTM**: Menghasilkan probabilitas untuk setiap kelas (negatif, netral, positif), sehingga kita mengambil argmax untuk mendapatkan label prediksi.
2. **Model SVM**: Langsung memberikan label kelas sebagai hasil dari prediksi.
3. **Model Random Forest**: Sama seperti SVM, model ini memberikan output berupa label kelas berdasarkan hasil voting dari berbagai pohon keputusan.

Prediksi ini akan menghasilkan label numerik (biasanya 0, 1, atau 2) yang kemudian bisa diinterpretasikan ke dalam kategori sentimen.

In [8]:
# Melakukan prediksi menggunakan model LSTM
# Hasil prediksi berupa probabilitas untuk setiap kelas, sehingga kita ambil nilai maksimum (argmax)
lstm_preds = np.argmax(lstm_model.predict(new_padded), axis=1)

# Melakukan prediksi menggunakan model SVM
# Model SVM secara langsung menghasilkan label prediksi dari data TF-IDF
svm_preds = svm_model.predict(new_tfidf)

# Melakukan prediksi menggunakan model Random Forest
# Sama seperti SVM, Random Forest memberikan prediksi berbasis data TF-IDF
rf_preds = rf_model.predict(new_tfidf)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 585ms/step


## Display Results

Setelah proses prediksi dilakukan, tahap selanjutnya adalah menampilkan hasil prediksi sentimen dari ketiga model yaitu LSTM, SVM, dan Random Forest. 

Langkah ini bertujuan untuk melihat dan membandingkan output prediksi dari masing-masing model terhadap input teks yang diberikan. Setiap label numerik hasil prediksi dikonversi menjadi label string ('negative', 'neutral', 'positive') agar lebih mudah dipahami. Hasil ini akan dicetak satu per satu untuk setiap teks input.

In [9]:
# Mapping label numerik ke label string agar hasil prediksi lebih mudah dibaca
label_map = {0: "negative", 1: "neutral", 2: "positive"}

# Menampilkan hasil prediksi dari ketiga model untuk setiap input teks
print("\nSentiment Prediction Results: ")
for i, text in enumerate(new_texts):
    print(f"Text: {text}")  # Menampilkan teks input
    print(f"LSTM Prediction: {label_map[lstm_preds[i]]}")  # Hasil prediksi model LSTM
    print(f"SVM Prediction: {label_map[svm_preds[i]]}")    # Hasil prediksi model SVM
    print(f"Random Forest Prediction: {label_map[rf_preds[i]]}")  # Hasil prediksi model RF
    print("-" * 50)  # Garis pemisah antar hasil


Sentiment Prediction Results: 
Text: Produk ini sangat bagus!
LSTM Prediction: positive
SVM Prediction: positive
Random Forest Prediction: positive
--------------------------------------------------
Text: Pelayanan sangat lambat dan mengecewakan.
LSTM Prediction: negative
SVM Prediction: negative
Random Forest Prediction: negative
--------------------------------------------------
Text: Harganya cukup mahal tapi kualitasnya biasa saja.
LSTM Prediction: neutral
SVM Prediction: neutral
Random Forest Prediction: neutral
--------------------------------------------------
